In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

from sklearn.cluster import KMeans 
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering

In [3]:
import sys

sys.path.append('../../src')



In [4]:
from base_exp_gen import generate_experiment as GEN_EXP
from clustering_algos import agglomerative_clustering, kmeans_clustering, box_clustering
from sklearn.model_selection import train_test_split as datasplit
from sklearn.ensemble import GradientBoostingRegressor as GDBR
from evaluations import computeATT_per_cluster, predict_cf, calculate_ite, get_homogeneity



In [5]:
np.random.seed(seed = 0)

In [6]:
pd.set_option("display.max_columns", None)


In [7]:
x_partition = [[i,i**2,2*i**4, 50000] for i in [2,4,5,10,12,20,25,33]]
def its(x):
    for i, (x1,x2,x3,x4) in enumerate(x[1:]):
        x[i+1][3]=(int(x[i][3]/(2*np.sqrt(x3/x[i][2]))))
    return x

In [8]:
a = its(x_partition)
exps = pd.DataFrame(a, columns = ['X-parts','Clusters', 'N', 'iters'])

In [9]:
name = 'box'
exps.index.name = name

In [10]:
exps['mean-hom'] = 0
exps['std-hom'] = 0

In [11]:
exps

,X-parts,Clusters,N,iters,mean-hom,std-hom
box,,,,,,
0,2,4,32,50000,0,0
1,4,16,512,6250,0,0
2,5,25,1250,2000,0,0
3,10,100,20000,250,0,0
4,12,144,41472,86,0,0
5,20,400,320000,15,0,0
6,25,625,781250,4,0,0
7,33,1089,2371842,1,0,0


In [12]:
import time

In [13]:
%%time
col_select = ['x0', 'x1']
gen = []
for i,row in exps.iterrows():
    c, n, iters = row.Clusters, row.N, row.iters
    homc = []
    print('running:',c,n)
    ss = time.time()
    for j in range(iters):
        ###gneerate data
        N,D = n,2
        kw_generate =  {'N':n, 'D':D,'f_gen_name': 'uniform_gen','low':0, 'high': 1}
        kw_cluster =  {'f_class_name':'circle_class', 'centers': [0.5, 0.5], 'r_small': 0.2, 'r_big':0.4}
        kw_treatment = {'f_treat_name' :'uniform_treat','choices':[0,1], 'probabilities':[0.5,0.5]}

        stats = np.array([[1, 0.75], [0, 0.75], [1, 0.75], [1, 0.75]])
        kw_outcome = {'f_outcome_name': 'outcome1','treatment':'Treatment', 'cls':'C', 'stats':stats}

        data = GEN_EXP(kw_generate, kw_cluster, kw_treatment, kw_outcome).dat
        data = calculate_ite(data.copy(), treatment = 'Treatment',
                     counterfactual = 'Ycf', outcome = 'Y', ite_name = 'ITE')
        
        ### cluster data
        data2,_ = box_clustering(data, clusters=c,col_select = col_select, cluster_name = 'A')
        ### calc hom
        res = computeATT_per_cluster(data2.copy(), hom_name = 'HOM',weight_names = 'W', 
                                     cluster_name  = "A", att = False)
        hom = (res['HOM']*res['W']).sum(axis = 0)
        homc.append(hom)
        
    ###get stats
    ee = time.time() - ss
    print('time:',ee)
    homcnp = np.array(homc)
    gen.append(homcnp)
    mean = np.mean(homcnp)
    std = np.std(homcnp)
    exps.loc[i,'mean-hom'] = mean
    exps.loc[i,'std-hom'] = std
    
    

running: 4 32
time: 1860.7741124629974
running: 16 512
time: 422.8585739135742
running: 25 1250
time: 179.69603419303894
running: 100 20000
time: 72.56229662895203
running: 144 41472
time: 35.71471667289734
running: 400 320000
time: 18.778613090515137
running: 625 781250
time: 8.631349086761475
running: 1089 2371842
time: 4.960816383361816
CPU times: user 43min 20s, sys: 6.09 s, total: 43min 26s
Wall time: 43min 24s


In [14]:
exps

,X-parts,Clusters,N,iters,mean-hom,std-hom
box,,,,,,
0,2,4,32,50000,0.671417,0.061389
1,4,16,512,6250,0.669695,0.015040
2,5,25,1250,2000,0.776799,0.010427
3,10,100,20000,250,0.937516,0.001523
4,12,144,41472,86,0.918619,0.001281
5,20,400,320000,15,0.962720,0.000296
6,25,625,781250,4,0.955020,0.000335
7,33,1089,2371842,1,0.973252,0.000000


In [15]:
from pathlib import Path



In [16]:
p = './homog/'
Path('./homog/').mkdir(parents = True, exist_ok = True)
exps.to_csv(p+name+'.csv')


### MISC

In [17]:
%%time
homlist = []
for i in range(100):
    N,D,c = 9**2,2,9
    kw_generate =  {'N':n, 'D':D,'f_gen_name': 'uniform_gen','low':0, 'high': 1}
    kw_cluster =  {'f_class_name':'circle_class', 'centers': [0.5, 0.5], 'r_small': 0.2, 'r_big':0.4}
    kw_treatment = {'f_treat_name' :'uniform_treat','choices':[0,1], 'probabilities':[0.5,0.5]}

    stats = np.array([[1, 0.75], [0, 0.75], [1, 0.75], [1, 0.75]])
    kw_outcome = {'f_outcome_name': 'outcome1','treatment':'Treatment', 'cls':'C', 'stats':stats}

    data3 = GEN_EXP(kw_generate, kw_cluster, kw_treatment, kw_outcome).dat
    data3 = calculate_ite(data3.copy(), treatment = 'Treatment',
                 counterfactual = 'Ycf', outcome = 'Y', ite_name = 'ITE')

    ### cluster data
    data3,_ = box_clustering(data3, clusters=c,col_select = col_select, cluster_name = 'A')
    ### calc hom
    res3 = computeATT_per_cluster(data3.copy(), hom_name = 'HOM',weight_names = 'W', 
                                 cluster_name  = "A", att = False)

    hom3 = (res3['HOM']*res3['W']).sum(axis = 0)
    homlist.append(hom3)

np.array(homlist).mean()

CPU times: user 3min 17s, sys: 48.4 s, total: 4min 6s
Wall time: 4min 9s


In [18]:
np.array(homlist).mean()

0.7290667464358925

In [19]:
res3

,A,ATT_CLUST,#CLUST_POINTS,HOM,W
0,0,0.217216,263379.0,0.782682,0.111044
1,1,0.622658,264300.0,0.620371,0.111432
2,2,0.216991,263927.0,0.781981,0.111275
3,3,0.619698,263204.0,0.617141,0.110970
4,4,0.047440,263143.0,0.950647,0.110945
5,5,0.620430,263567.0,0.619797,0.111123
6,6,0.217108,264353.0,0.783324,0.111455
7,7,0.619409,262726.0,0.618724,0.110769
8,8,0.217943,263243.0,0.782714,0.110987


In [20]:
hom3 = (res3['HOM']*res3['W']).sum(axis = 0)


In [21]:
sc.special.comb(4,2)

NameError: name 'sc' is not defined

In [ ]:
import scipy as sc


In [ ]:
mu = 0.63
ss = 0
N = 16
for k in range(N+1):
    prod= sc.special.comb(N,k)*((1/4)**k)*(3/4)**(N-k)
    sumi = 0
    for i in range(k+1):
        sumi += max(i,k-i)*sc.special.comb(k,i)*(mu**i)*(1-mu)**(k-i)
        
    ss += prod*sumi
        

In [ ]:
ss/4

In [ ]:
sss = 0
k= 4
mu = 0.623
for i in range(k+1):
    prod= max(k-i,i)*sc.special.comb(k,i)*((mu)**i)*(1-mu)**(k-i)
    sss += prod

In [ ]:
sss/k

In [ ]:
ss/4